# Lab 3: Extending Logistic Regression
## by Michael Doherty, Leilani Guzman, and Carson Pittman

## 1. Business Understanding

## 2. Data Preparation

## 3. Modeling

## 4. Deployment

## 5. Exceptional Work (rename to what we actually end up doing)